In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from urllib.parse import quote
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

## filePath 설정

In [2]:
import os

path = './지마켓'
os.makedirs(path)

for i in range(2,4) :
    folder_name = f'./지마켓/depth{i}'
    os.makedirs(folder_name)

## 수집함수 정의

In [ ]:
# 카테고리마다 수집
def crawl_gm(depth = 2, limit = 11) :
    if depth == 2 :
        # 상위 카테고리
        gm_df = pd.DataFrame()
        for i in range(1,201) :
            driver.get('http://browse.gmarket.co.kr/list?category=200002920&gate_id=TVNUDGAX-O6HJ-ODF6-934Y-WI68HC1NF52L&s=8&k=0&p={}'.format(i))
                        # 페이지별로 페이지url 정의
            time.sleep(3)

            html = driver.page_source
                # 링크의 html 정보 스크랩핑
            soup = BeautifulSoup(html, 'html.parser')
                # html에 있는 문자열만 모두 찾아서 출력
            
            # url 파서
            urls = soup.select('a.link__item')
            lst_list = []
            cnt = 0 
            for url in urls :
                if cnt%2 == 0 :
                        # href 중 허수 제거 (홀수번째 값만 파서)
                    cnt += 1
                    lst_list.append(url.attrs['href'])
                else: 
                    cnt+= 1

            # 상품명 파서
            prdt_nm = soup.select('span.text__item')
            tst_list = []
            for prdt in prdt_nm :
                tst_list.append(prdt.text.strip())
                    # text 값만 가져옴

            new_df = pd.DataFrame(lst_list,tst_list).reset_index()
            new_df['페이지'] = len(tst_list)*[i]
            new_df.columns = ['상품명','url','페이지']
            gm_df = pd.concat([gm_df,new_df])

            gm_df.drop_duplicates(['url'],keep = 'last', inplace = True)
                # url 중복값 중 가장 마지막값만 남겨두고 나머지는 삭제
            gm_df = gm_df.reset_index(drop = True)
            print(f'{i}번째 페이지 수집 완료, 수집 개수 : {len(lst_list),len(prdt_nm)}')
        gm_df.to_csv(f'G:/다른 컴퓨터/내 MacBook Air/MC/depth연구/3_지마켓(판매인기순)/세탁기/depth2/세탁기(지마켓,{len(gm_df)}개).csv', index = False, encoding = 'utf-8-sig')
        
    else :
        # 하위 카테고리
        gm_df = pd.DataFrame()
        urls = f'{driver.current_url}'
        for i in range(1,limit) :            
            driver.get(f'{driver.current_url}&k=0&p={i}')
                # 현재 url에 추가적인 쿼리 매개변수를 포합한 새로운 url로 이동
            time.sleep(3)

            html = driver.page_source
                    # 현재 웹페이지의 HTML 소스코드를 가져옴 -> 파싱
            soup = BeautifulSoup(html, 'html.parser')
            
            urls = soup.select('a.link__item')
                                    # 'a' 태그 중 클래스가 'link__item'인 요소 선택
            lst_list = []

            cnt = 0 
            for url in urls :
                if cnt%2 == 0 :
                        # 짝수번째 url만 수집
                    cnt += 1
                    lst_list.append(url.attrs['href'])
                else: 
                    cnt+= 1
                    
            # 상품명
            prdt_nm = soup.select('span.text__item')
                                    # 상품명 : 'span' 태그 중 클래스가 'text__item'인 요소들
            tst_list = []
            for prdt in prdt_nm :
                tst_list.append(prdt.text.strip())

            new_df = pd.DataFrame(lst_list,tst_list).reset_index()
            new_df['페이지'] = len(tst_list)*[i]
            new_df.columns = ['상품명','url','페이지']
            gm_df = pd.concat([gm_df,new_df])

            gm_df.drop_duplicates(['url'],keep = 'last', inplace = True)
            gm_df = gm_df.reset_index(drop = True)
            print(f'{i}번째 페이지 수집 완료, 수집 개수 : {len(lst_list),len(prdt_nm)}')
        return gm_df